<a href="https://colab.research.google.com/github/1shikapandey/Text-Summarization-with-BART/blob/main/Text_Summarization_with_BART_Abstractive_NLP_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Step 1: Install Dependencies

In [ ]:
!pip install transformers sentencepiece evaluate rouge_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.7 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=83a53aac04437fd62cc812f91494398dbf3c1f049b7c9cc66955e0a3953447ee
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


## Step 2: Import Libraries

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

## Step 3: Load Pre-trained Model

In [ ]:
model_name = "facebook/bart-large-cnn"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

Device set to use cpu


## Step 4: Input Text

In [ ]:
article = """
Artificial Intelligence (AI) is rapidly transforming various industries.
From healthcare and finance to education and transportation, AI is driving innovations
that were once considered science fiction. In healthcare, AI assists doctors in diagnosing
diseases faster and more accurately. In finance, AI algorithms analyze massive datasets
to detect fraud and predict market trends. In education, AI personalizes learning experiences
for students. However, alongside these advancements, ethical concerns such as job displacement,
privacy, and bias in AI systems have also emerged. Policymakers and researchers are working
together to ensure AI develops responsibly and benefits society as a whole.
"""

## Step 5: Tokenize Input

In [ ]:
inputs = tokenizer([article], max_length=1024, truncation=True, return_tensors="pt")

## Step 6: Generate Summary

In [ ]:
summary_ids = model.generate(
    inputs["input_ids"],
    max_length=80,
    min_length=50,
    length_penalty=2.0,
    num_beams=4,
    early_stopping=True
)

summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

## Step 7: Display Results

In [ ]:
import textwrap

print("Original Text:\n")
print(textwrap.fill(article, width=100))

print("\nGenerated Summary:\n")
print(textwrap.fill(summary, width=100))

Original Text:

 Artificial Intelligence (AI) is rapidly transforming various industries.  From healthcare and
finance to education and transportation, AI is driving innovations  that were once considered
science fiction. In healthcare, AI assists doctors in diagnosing  diseases faster and more
accurately. In finance, AI algorithms analyze massive datasets  to detect fraud and predict market
trends. In education, AI personalizes learning experiences  for students. However, alongside these
advancements, ethical concerns such as job displacement,  privacy, and bias in AI systems have also
emerged. Policymakers and researchers are working  together to ensure AI develops responsibly and
benefits society as a whole.

Generated Summary:

Artificial Intelligence (AI) is rapidly transforming various industries. In healthcare, AI assists
doctors in diagnosing diseases faster and more accurately. In finance, AI algorithms analyze massive
datasets to detect fraud and predict market trends. In edu

## Step 8: Evaluate Coherence & Relevance (ROUGE)


In [ ]:
import evaluate

rouge = evaluate.load("rouge")
results = rouge.compute(predictions=[summary], references=[article])
print("\nROUGE Evaluation Metrics:")
for k, v in results.items():
    print(f"{k}: {v:.4f}")


ROUGE Evaluation Metrics:
rouge1: 0.6939
rouge2: 0.6621
rougeL: 0.6939
rougeLsum: 0.6939
